<a href="https://colab.research.google.com/github/GenAIHub/agents-workshop/blob/main/02_chatbot_with_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Enhancing the Chatbot with Tools

To answer questions beyond the chatbot's LLM built-in knowledge, we'll add a web search tool to help the bot find relevant information online and give better responses.

#### Requirements
Before we start, install the requirements to use the [Tavily Search Engine](https://python.langchain.com/v0.2/docs/integrations/tools/tavily_search/), and set your [TAVILY_API_KEY](https://tavily.com/).

In [ ]:
%%capture --no-stderr
%pip install -U tavily-python
%pip install -U langchain langchain-core langchain-openai langchain-community 
%pip install -U langgraph

Set API keys

In [ ]:
import os

# Set environment variables
os.environ["AZURE_OPENAI_API_KEY"] = ""
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://app-ads-sbx-openai-sw.openai.azure.com"
os.environ["AZURE_OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = "gpt-4o"

os.environ["TAVILY_API_KEY"] = ""

Initialize the Azure LLM

In [ ]:
from langchain_openai import AzureChatOpenAI

# Fetching environment variables
api_key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

if not all([api_key, endpoint, api_version, deployment_name]):
    raise ValueError("One or more environment variables are missing.")

# Initialize the Azure LLM
llm = AzureChatOpenAI(
    openai_api_key=api_key,
    azure_endpoint=endpoint,
    azure_deployment=deployment_name,
    openai_api_version=api_version,
)

Adding a Web Search Tool <br>
<br>
Let's add a web search tool to our chatbot to enable it to search the web for information. We can integrate this using an external API or a custom function. <br>
In this case, we will make use of the **Tavily API** to search the web. <br>
For simplicity, LangChain offers a pre-built tool that we can use out-of-the-box!

In [ ]:
# Import the Tavily search tool
from langchain_community.tools.tavily_search import TavilySearchResults

# Initialize the Tavily tool
# We limit the max results Tavily returns to 2
search_tool = TavilySearchResults(max_results=2)
tools = [search_tool]

# Bind the tools to the LLM
llm_with_tools = llm.bind_tools(tools)

In [ ]:
# Invoke the Tavily search tool with a sample query to see how it works.
# This simulates the chatbot using the tool to find information about "nodes" in LangGraph.
search_tool.invoke("What's a 'node' in LangGraph?")

The results are page summaries our chat bot can use to answer questions.

Creating a `StateGraph`. 

In [ ]:
# Define the state and graph, similar to last notebook
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

# Define the chatbot function
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# Add the chatbot node to the graph
graph_builder.add_node("chatbot", chatbot)

graph_builder.set_entry_point("chatbot")

In [ ]:
from langgraph.prebuilt import ToolNode, tools_condition

# Use a prebuilt ToolNode (offered by LangGraph itself) to handle tool calls.
# ToolNode will call the correct tool based on the output of our chatbot!
tool_node = ToolNode(tools=[search_tool])

# Add the toolnode to the graph
graph_builder.add_node("tools", tool_node)

# Define conditional edges to manage the flow of the graph.
# The condition will route to "tools" if tool calls are present, 
# and to "__end__" if no tool calls are made.
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

# Each time a tool is called, we return to the chatbot to decide the next step.
graph_builder.add_edge("tools", "chatbot")

# Compile the graph
graph = graph_builder.compile()

Let's visualize the graph we've built. 

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

Now we can ask the bot questions outside its training data. <br>
Try asking it what the current weather in LA is!

In [ ]:
from langchain_core.messages import BaseMessage

while True:
    user_input = input("\n\nUser: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break
    for event in graph.stream({"messages": [("user", user_input)]}):
        for value in event.values():
            if isinstance(value["messages"][-1], BaseMessage):
                print("Assistant:", value["messages"][-1].content)

### **Built your own tool**

In this section we will built our own image generator tool with 'DALL-E'. <br>

In [ ]:
from IPython.display import Image, display
import json

from openai import AzureOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import StructuredTool

client = AzureOpenAI(
    api_version="2024-02-01",
    api_key=api_key,
    azure_endpoint=endpoint,
    azure_deployment="dall-e-3"
)

# The function to be called whenever the tool is invoked
def generate_dalle_image(prompt: str):

    # Generate an image with DALL-E
    result = client.images.generate(
        model="dalle3",  
        prompt=prompt,
        n=1
    )

    json_response = json.loads(result.model_dump_json())

    # Retrieve the generated image
    image_url = json_response["data"][0]["url"]  # extract image URL from response

    display(Image(url=image_url))

    return 'Image generated successfully'

# A class describing the input arguments of the function behind our tool
class ImageGeneratorInput(BaseModel):
    prompt: str = Field(
        description="The detailed prompt that will be given to the dall-e-3 model to generate the image."
    )

# Create the tool
# The description is very important as it provides the LLM with the knowledge of WHEN to call the tool
# The args_schema is very important as it provides the LLM with the knowledge of HOW to call the tool
image_generator_tool = StructuredTool.from_function(
    func=generate_dalle_image,
    name="image_generator",
    description="Generate an image given a detailed description through the use of the dall-e-3 model.",
    args_schema=ImageGeneratorInput,
    return_direct=True
)

#### **Question (optional):**

The following code builds the new graph with our new tool. <br>
However, it does not work! <br>
Something is missing, can you figure out what important step was looked over?

In [ ]:
# Add the image_generator_tool to our tools
tools = [search_tool, image_generator_tool]

# -----------------------------------------------

new_graph_builder = StateGraph(State)

# Add the chatbot node to the graph
new_graph_builder.add_node("chatbot", chatbot)
new_graph_builder.set_entry_point("chatbot")

# Use a prebuilt ToolNode (offered by LangGraph itself) to handle tool calls.
# ToolNode will call the correct tool based on the output of our chatbot!
tool_node = ToolNode(tools=tools)

# Add the toolnode to the graph
new_graph_builder.add_node("tools", tool_node)

# Define conditional edges to manage the flow of the graph.
# The condition will route to "tools" if tool calls are present, 
# and to "__end__" if no tool calls are made.
new_graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

# Each time a tool is called, we return to the chatbot to decide the next step.
new_graph_builder.add_edge("tools", "chatbot")

# Compile the graph
new_graph = new_graph_builder.compile()

#### **Solution**:

Recall that chatbot makes use of **'llm_with_tools'**! <br>
Our LLM has not been updated yet to be bound to our newly updated list of tools. <br>
As a result, the LLM has no knowledge of this new tool and will never call it. <br>
All we did in the code above was ensure that our graph knew of its existence.<br>

In [ ]:
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

tools = [search_tool, image_generator_tool]

llm_with_tools = llm.bind_tools(tools)

Try out the new tool! <br>
Ask the system to generate an image.

In [ ]:
from langchain_core.messages import BaseMessage

while True:
    user_input = input("\n\nUser: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break
    for event in new_graph.stream({"messages": [("user", user_input)]}):
        for value in event.values():
            if isinstance(value["messages"][-1], BaseMessage):
                print("Assistant:", value["messages"][-1].content)

**Congrats!** You've created a conversational agent in langgraph that can use a search engine to retrieve updated information when needed. 